## Linear Regression สำหรับทำนายค่า Attack ของ Pokemon
* Name : Supawit Kansompoj 57011272
* Subject : Introduction To Data Analytics
* Semester : 2/2560
* Date : 12/02/61

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
df = pd.read_csv("../input/pokemon.csv")

### Data Extraction

In [ ]:
df2 = df.select_dtypes(include=['float64','int64'])
# all index that calcualtable
iter_index = [i for i in list(df2) if i != 'attack']

### The number of null value

In [ ]:
for i in iter_index:
    if df[i].isnull().values.any():
        print(df[i].name)
        print(df[i].isnull().sum())

### Data cleaning

In [ ]:
for i in iter_index:
   if df[i].isnull().values.any():
        df[i].fillna(df[i].mean(), inplace=True)

### Check null value

In [ ]:
df[iter_index].isnull().values.any()

### Correlation

In [ ]:
def corr_filter(x):
    return x.abs()
df_corr = df[iter_index + ['attack']].corr().apply(corr_filter, axis=1)
x = df_corr['attack'].sort_values(ascending=False)
x

In [ ]:
plt.subplots(figsize=(20,15))
sns.heatmap(df2.corr())

### Features Selection

In [ ]:
features = ['base_total','defense','sp_attack']
X = df[features]
y = df['attack']

### Features
1. base_total - มีค่า correlation เมื่อเทียบกับ attack สูง และไม่มี null value ในข้อมูล
1. defense - เมื่อค่า correlation ของแต่ละ feature เท่าๆกันเลยคิดว่า ค่า defense น่าจะมีผลกับ attack ของ pokemon และไม่มี null value ในข้อมูล
1. sp_attack - เมื่อค่า correlation ของแต่ละ feature เท่าๆกันเลยคิดว่า ค่า ap_attack น่าจะมีผลกับ attack ของ pokemon และไม่มี null value ในข้อมูล

### Histrogram

In [ ]:
for i in features + ['attack']:
    plt.figure()
    sns.distplot(df[i])

### Box-plot

In [ ]:
for i in features + ['attack']:
    plt.figure()
    sns.set_style("whitegrid")
    sns.boxplot(x=df[i])

### Scatter-plot

In [ ]:
for i in features:
    plt.figure()
    sns.regplot(x=df[i], y=y, fit_reg=False)

### Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

kf = KFold(n_splits=10)
kf.get_n_splits(X)

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

# Linear Regression
lm = LinearRegression()
lm.fit(X_train,y_train)
coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
predictions = lm.predict(X_test)

print(lm.intercept_)
print(coeff_df)

### Prediction equeation
y = base_total x (0.315208) + defense x (-0.117763) + sp_attack x (-0.492646) + (-13.5759632621)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

### 10-Fold Crossvalidation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn import metrics

kf = KFold(n_splits=10)
kf.get_n_splits(X)

In [ ]:
loss = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    # Linear Regression
    lm = LinearRegression()
    lm.fit(X_train,y_train)
    coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
    predictions = lm.predict(X_test)
    loss.append([
        metrics.mean_absolute_error(y_test, predictions),
        metrics.mean_squared_error(y_test, predictions),
        np.sqrt(metrics.mean_squared_error(y_test, predictions))
    ])

### Metrics

In [ ]:
loss = np.array(loss)
mean_loss = loss.mean(axis=0)
print('MAE:', mean_loss[0])
print('MSE:', mean_loss[1])
print('RMSE:', mean_loss[2])